In [1]:
from datasets import load_dataset
import polars as pl
from IPython.display import display, Markdown, Latex
from collections import defaultdict
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from rich.syntax import Syntax
from rich.console import Console
from enum import Enum
from functools import partial
%load_ext autoreload
%autoreload 2

/Users/brian/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
sys.path.append('..')
from evaluation.code_execution import evaluate_sample_codecontests, evaluate_python_code_exec

In [3]:
cc_dataset = load_dataset("deepmind/code_contests")["train"]

In [4]:
cc_dataset

Dataset({
    features: ['name', 'description', 'public_tests', 'private_tests', 'generated_tests', 'source', 'difficulty', 'solutions', 'incorrect_solutions', 'cf_contest_id', 'cf_index', 'cf_points', 'cf_rating', 'cf_tags', 'is_description_translated', 'untranslated_description', 'time_limit', 'memory_limit_bytes', 'input_file', 'output_file'],
    num_rows: 13328
})

In [58]:
console = Console()

first_n_samples = {}


class Language(Enum):
    UNKNOWN = 0
    PYTHON = 1
    CPP = 2
    PYTHON3 = 3
    JAVA = 4


def print_code_snippet(snippet: str, console: Console):
    formatted_snippet = Syntax(
        snippet,
        "python",
        theme="monokai",
        line_numbers=True,
    )
    console.print(formatted_snippet)


sample_lang_counts = defaultdict(int)
n_samples_evaluated = 0
n_errors = 0
for i, sample in enumerate(tqdm(cc_dataset)):
    for k, v in sample.items():  # type: ignore
        if k not in first_n_samples:
            first_n_samples[k] = []
        first_n_samples[k].append(v)
    description = sample["description"]
    test_inputs = sample["public_tests"]["input"] + sample["private_tests"]["input"]
    test_outputs = sample["public_tests"]["output"] + sample["private_tests"]["output"]
    for lang_txt, solution_code in zip(
        sample["solutions"]["language"], sample["solutions"]["solution"]
    ):
        language = Language(lang_txt)
        sample_lang_counts[language] += 1
        if language != Language.PYTHON3:
            continue

        # console.print(description)
        # print_code_snippet(solution_code, console)
        n_samples_evaluated += 1
        for inputs, outputs in zip(test_inputs, test_outputs):
            # each time input is called, it will return the next input value
            out = evaluate_python_code_exec(solution_code, inputs)
            print(out)


    # print(n_errors, n_samples_evaluated)

  0%|          | 0/13328 [00:00<?, ?it/s]Traceback (most recent call last):
  File "/Users/brian/smolmodels/notebooks/../evaluation/code_execution.py", line 217, in evaluate_python_code_exec
    exec(code_to_run, exec_globals)
  File "<string>", line 5, in <module>
TypeError: evaluate_python_code_exec.<locals>.<lambda>() takes 0 positional arguments but 1 was given
Traceback (most recent call last):
  File "/Users/brian/smolmodels/notebooks/../evaluation/code_execution.py", line 217, in evaluate_python_code_exec
    exec(code_to_run, exec_globals)
  File "<string>", line 5, in <module>
TypeError: evaluate_python_code_exec.<locals>.<lambda>() takes 0 positional arguments but 1 was given
Traceback (most recent call last):
  File "/Users/brian/smolmodels/notebooks/../evaluation/code_execution.py", line 217, in evaluate_python_code_exec
    exec(code_to_run, exec_globals)
  File "<string>", line 5, in <module>
TypeError: evaluate_python_code_exec.<locals>.<lambda>() takes 0 positional argu

['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']

Traceback (most recent call last):
  File "/Users/brian/smolmodels/notebooks/../evaluation/code_execution.py", line 217, in evaluate_python_code_exec
    exec(code_to_run, exec_globals)
  File "<string>", line 1, in <module>
TypeError: evaluate_python_code_exec.<locals>._retrieve_input() takes 0 positional arguments but 1 was given
Traceback (most recent call last):
  File "/Users/brian/smolmodels/notebooks/../evaluation/code_execution.py", line 217, in evaluate_python_code_exec
    exec(code_to_run, exec_globals)
  File "<string>", line 1, in <module>
TypeError: evaluate_python_code_exec.<locals>._retrieve_input() takes 0 positional arguments but 1 was given
Traceback (most recent call last):
  File "/Users/brian/smolmodels/notebooks/../evaluation/code_execution.py", line 217, in evaluate_python_code_exec
    exec(code_to_run, exec_globals)
  File "<string>", line 1, in <module>
TypeError: evaluate_python_code_exec.<locals>._retrieve_input() takes 0 positional arguments but 1 was give

['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']

Traceback (most recent call last):
  File "/Users/brian/smolmodels/notebooks/../evaluation/code_execution.py", line 217, in evaluate_python_code_exec
    exec(code_to_run, exec_globals)
  File "<string>", line 82, in <module>
  File "<string>", line 74, in __init__
  File "<string>", line 42, in __init__
io.UnsupportedOperation: fileno
Traceback (most recent call last):
  File "/Users/brian/smolmodels/notebooks/../evaluation/code_execution.py", line 217, in evaluate_python_code_exec
    exec(code_to_run, exec_globals)
  File "<string>", line 82, in <module>
  File "<string>", line 74, in __init__
  File "<string>", line 42, in __init__
io.UnsupportedOperation: fileno
Traceback (most recent call last):
  File "/Users/brian/smolmodels/notebooks/../evaluation/code_execution.py", line 217, in evaluate_python_code_exec
    exec(code_to_run, exec_globals)
  File "<string>", line 82, in <module>
  File "<string>", line 74, in __init__
  File "<string>", line 42, in __init__
io.UnsupportedOper

...

Traceback (most recent call last):
  File "/Users/brian/smolmodels/notebooks/../evaluation/code_execution.py", line 217, in evaluate_python_code_exec
    exec(code_to_run, exec_globals)
  File "<string>", line 22, in <module>
  File "<string>", line 6, in main
  File "/Users/brian/smolmodels/notebooks/../evaluation/code_execution.py", line 159, in readlines
    raise IOError
OSError
Traceback (most recent call last):
  File "/Users/brian/smolmodels/notebooks/../evaluation/code_execution.py", line 217, in evaluate_python_code_exec
    exec(code_to_run, exec_globals)
  File "<string>", line 22, in <module>
  File "<string>", line 6, in main
  File "/Users/brian/smolmodels/notebooks/../evaluation/code_execution.py", line 159, in readlines
    raise IOError
OSError
Traceback (most recent call last):
  File "/Users/brian/smolmodels/notebooks/../evaluation/code_execution.py", line 217, in evaluate_python_code_exec
    exec(code_to_run, exec_globals)
  File "<string>", line 22, in <module>
  

['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']

Traceback (most recent call last):
  File "/Users/brian/smolmodels/notebooks/../evaluation/code_execution.py", line 217, in evaluate_python_code_exec
    exec(code_to_run, exec_globals)
  File "<string>", line 18, in <module>
  File "<string>", line 13, in read_int
  File "/Users/brian/smolmodels/notebooks/../evaluation/code_execution.py", line 156, in readline
    raise IOError
OSError
Traceback (most recent call last):
  File "/Users/brian/smolmodels/notebooks/../evaluation/code_execution.py", line 217, in evaluate_python_code_exec
    exec(code_to_run, exec_globals)
  File "<string>", line 18, in <module>
  File "<string>", line 13, in read_int
  File "/Users/brian/smolmodels/notebooks/../evaluation/code_execution.py", line 156, in readline
    raise IOError
OSError
Traceback (most recent call last):
  File "/Users/brian/smolmodels/notebooks/../evaluation/code_execution.py", line 217, in evaluate_python_code_exec
    exec(code_to_run, exec_globals)
  File "<string>", line 18, in <mo

['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['passed']
['p

SystemExit: 

In [ ]:
sample_lang_counts